In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm.notebook import tqdm

In [ ]:
# Load our data.
df = pd.read_csv("data/Normalized_Purchase_Card_Transactions.csv")
df_agencies = pd.read_csv("data/List_Agencies.csv")
df_vendors = pd.read_csv("data/List_Vendors.csv")

In [ ]:
blank = np.zeros((df['AGENCY_NUM'].max() + 1, df['VENDOR_NAME_NUM'].max() + 1))

for index, row in tqdm(df.iterrows(), total = df.shape[0]):
    blank[row['AGENCY_NUM'], row['VENDOR_NAME_NUM']] += row['TRANSACTION_AMOUNT_NORM']

np.count_nonzero(blank)

In [ ]:
wordCounts = pd.DataFrame(blank)

bags = {i: df[df['AGENCY_NUM'] == i]['VENDOR_NAME_NUM'].unique() for i in df['AGENCY_NUM'].unique()}

In [ ]:
def tf(counts):
    tfDF = pd.DataFrame()
    
    for index, row in counts.iterrows():
        tfDF[index] = row.div(row.sum())
        
    return tfDF

tf = pd.DataFrame(tf(wordCounts)).T

In [ ]:
idf = tf.apply(lambda x: np.log((df['AGENCY_NUM'].max() + 1) / np.count_nonzero(x.to_numpy())), axis=0)

In [ ]:
prod = tf*idf

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(prod).astype("float")
for i in range(len(similarity)):
    similarity[i][i] = 0

In [ ]:
mostsimilar = np.argmax(similarity, axis=1)
mostsimilar_verbose = np.array([df_agencies.iloc[i][1] for i in mostsimilar])
mostsimilar_vendors = np.array([bags[i] for i in mostsimilar], dtype = object)

In [ ]:
def summarize(agency = 0):
    """
    Gives brief information on the agency and its most similar agency.
    """
    agency_bag = bags[agency]
    agency_bag_verbose = np.array([df_vendors.iloc[i][1] for i in agency_bag])
    similar_agency_bag = bags[mostsimilar[agency]]
    similar_agency_bag_verbose = np.array([df_vendors.iloc[i][1] for i in similar_agency_bag])
    
    print("Selected Agency:", df_agencies.iloc[agency][1])
    print("Most Similar Agency:", mostsimilar_verbose[agency])
    print()
    print("Agency Vendors:", agency_bag_verbose)
    print()
    print("Most Similar Agency Vendors:", similar_agency_bag_verbose)

In [ ]:
summarize(5)